In [6]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

## Importing csv files to dataframes

In [32]:
#Load in beers file from Kaggle
beers_file = "Resources/beers.csv"
beers_df = pd.read_csv(beers_file).fillna(0).rename(columns={"Unnamed: 0":"index", "name":"beer_name"})
beers_df

,index,abv,ibu,id,beer_name,style,brewery_id,ounces
0,0,0.050,0.0,1436,Pub Beer,American Pale Lager,408,12.0
1,1,0.066,0.0,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,2,0.071,0.0,2264,Rise of the Phoenix,American IPA,177,12.0
3,3,0.090,0.0,2263,Sinister,American Double / Imperial IPA,177,12.0
4,4,0.075,0.0,2262,Sex and Candy,American IPA,177,12.0
5,5,0.077,0.0,2261,Black Exodus,Oatmeal Stout,177,12.0
6,6,0.045,0.0,2260,Lake Street Express,American Pale Ale (APA),177,12.0
7,7,0.065,0.0,2259,Foreman,American Porter,177,12.0
8,8,0.055,0.0,2258,Jade,American Pale Ale (APA),177,12.0
9,9,0.086,0.0,2131,Cone Crusher,American Double / Imperial IPA,177,12.0


In [33]:
#Load in breweries file from Kaggle
breweries_file = "Resources/breweries.csv"
breweries_df = pd.read_csv(breweries_file).rename(columns={"Unnamed: 0":"brewery_id", "name":"brewery_name"})
breweries_df

,brewery_id,brewery_name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA
5,5,COAST Brewing Company,Charleston,SC
6,6,Great Divide Brewing Company,Denver,CO
7,7,Tapistry Brewing,Bridgman,MI
8,8,Big Lake Brewing,Holland,MI
9,9,The Mitten Brewing Company,Grand Rapids,MI


## Windows Path

In [10]:
 executable_path = {'executable_path': 'chromedriver.exe'}
 browser = Browser('chrome', **executable_path, headless=False)

## Mac Path

In [5]:
#executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
#browser = Browser('chrome', **executable_path, headless=False)

## Scraping Data from Brewers Association Website

In [11]:
url = 'https://www.brewersassociation.org/statistics-and-data/state-craft-beer-stats/'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [12]:
#State cards with data to scrape
states_cards = soup.find('section', class_='site-content wide')
states = soup.find_all('div', class_='stat-container')


#Set up lists to append into
state_list = []
million_dollars_list = []
breweries_per_capita_list = []
barrels_produced_list = []
gallons_per_adult_list = []


In [34]:
#Loop to scrape data from the brewers association website for each state information card
for state in states:
    state_name = state.find('h1').text.strip()
    state_list.append(state_name)
    million_dollars = state.find('span', class_='total').find('span', class_='count').text.strip()
    million_dollars_list.append(million_dollars)
    breweries_per_capita = state.find('span', class_="bpc").find('span', class_= 'count').text.strip()
    breweries_per_capita_list.append(breweries_per_capita)
    barrels_produced = state.find(id ='production').find('span', class_='total').find('span', class_='count').text.strip()
    barrels_produced_list.append(barrels_produced)
    gallons_per_adult = state.find(id = 'production').find('span', class_='per-capita').find('span', class_='count').text.strip()
    gallons_per_adult_list.append(gallons_per_adult)

In [14]:
#state_list

In [15]:
#million_dollars_list

In [16]:
#breweries_per_capita_list

In [17]:
#barrels_produced_list

In [18]:
#gallons_per_adult_list

In [35]:
#renaming DataFrame columns
brewers_association_data = pd.DataFrame(
    {'state': state_list,
     'millions_sales': million_dollars_list,
     'breweries_per_capita': breweries_per_capita_list,
     'barrels_produced': barrels_produced_list,
     'gallons_per_adult': gallons_per_adult_list
    })


In [20]:
brewers_association_data

,state,millions_sales,breweries_per_capita,barrels_produced,gallons_per_adult
0,Alabama,758,1.1,"71,894",0.6
1,Alaska,326,7.8,"210,063",12.1
2,Arizona,"1,147",2.2,"173,427",1
3,Arkansas,838,1.8,"45,720",0.6
4,California,"9,014",2.9,"3,421,295",3.6
5,Colorado,"3,285",9.2,"1,522,834",11
6,Connecticut,753,3.2,"213,676",2.4
7,Delaware,388,3.7,"298,706",12.5
8,District of Columbia,213,2.4,"33,857",1.9
9,Florida,"3,625",1.7,"1,373,558",2.6


## Loading into a sqlite database

In [21]:
engine = create_engine('sqlite:///Resources/brew.sqlite')
Base.metadata.create_all(engine)
session = Session(engine)

In [22]:
# Confirm tables
engine.execute('DROP TABLE IF EXISTS beers')
engine.execute('DROP TABLE IF EXISTS breweries')
engine.execute('DROP TABLE IF EXISTS association')

beers_df.to_sql('beers', con=engine, if_exists='append', index=True)
breweries_df.to_sql('breweries', con=engine, if_exists='append', index=True)
brewers_association_data.to_sql('association', con=engine, if_exists='append', index=True)

In [23]:
#Calculate summary level statisics on DataFrame
csv_df = pd.DataFrame(engine.execute
                            ('SELECT r.state, COUNT(DISTINCT r.brewery_id), COUNT(DISTINCT e.beer_name), '\
                             'COUNT(DISTINCT e.style), COUNT(DISTINCT r.city), '\
                             'AVG(e.abv), AVG(e.ibu) FROM beers e '\
                             'INNER JOIN breweries r ON r.brewery_id=e.brewery_id '\
                             'GROUP BY r.state ORDER BY r.state;'
                            ))

In [24]:
# combined_df = pd.DataFrame(engine.execute('SELECT * FROM brews'))
csv_df = csv_df.rename(columns={0:"state", 1:"brewery_count", 2:"beer_counts",\
    3:"style_counts", 4:"cities", 5:"average_abv", 6:"average_ibu", 7:"style", 8:"ounces"})
csv_df

,state,brewery_count,beer_counts,style_counts,cities,average_abv,average_ibu
0,AK,7,25,14,4,0.055640,27.800000
1,AL,3,10,8,3,0.062000,46.100000
2,AR,2,5,5,2,0.052000,7.800000
3,AZ,11,46,19,8,0.056383,17.978723
4,CA,39,173,46,24,0.060749,34.142077
5,CO,47,252,61,27,0.059785,26.132075
6,CT,8,27,15,7,0.061074,9.074074
7,DC,1,8,8,1,0.065625,27.625000
8,DE,2,2,2,2,0.027500,26.000000
9,FL,15,57,19,11,0.057879,29.827586


In [30]:
#Changing abbreviated state names to full state names
csv_df["state"]= (["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","District of Columbia","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"])

In [26]:
engine.execute('DROP TABLE IF EXISTS brews')
csv_df.to_sql('brews', con=engine, if_exists='append', index=True)

In [31]:
#Combine dataframes and calculate summary level statistics
combined_df = pd.DataFrame(engine.execute
                            ('SELECT b.state, b.brewery_count, b.beer_counts, b.style_counts, '\
                             'b.cities, b.average_abv, b.average_ibu,'\
                             'a.breweries_per_capita, a.millions_sales, a.barrels_produced '\
                             'FROM brews b '\
                             'INNER JOIN association a ON b.state=a.state '\
                             'GROUP BY b.state ORDER BY b.state;'
                            ))
combined_df.rename(columns={0:"state", 1:"brewery_count", 2:"beer_counts", 3:"style_counts", 
                            4:"cities", 5:"average_abv", 6:"average_ibu", 7:"breweries_per_capita", 
                            8:"sales_in_millions", 9:"barrels_produced"})

,state,brewery_count,beer_counts,style_counts,cities,average_abv,average_ibu,breweries_per_capita,sales_in_millions,barrels_produced
0,Alabama,7,25,14,4,0.055640,27.800000,1.1,758,"71,894"
1,Alaska,3,10,8,3,0.062000,46.100000,7.8,326,"210,063"
2,Arizona,2,5,5,2,0.052000,7.800000,2.2,"1,147","173,427"
3,Arkansas,11,46,19,8,0.056383,17.978723,1.8,838,"45,720"
4,California,39,173,46,24,0.060749,34.142077,2.9,"9,014","3,421,295"
5,Colorado,47,252,61,27,0.059785,26.132075,9.2,"3,285","1,522,834"
6,Connecticut,8,27,15,7,0.061074,9.074074,3.2,753,"213,676"
7,Delaware,2,2,2,2,0.027500,26.000000,3.7,388,"298,706"
8,District of Columbia,1,8,8,1,0.065625,27.625000,2.4,213,"33,857"
9,Florida,15,57,19,11,0.057879,29.827586,1.7,"3,625","1,373,558"


In [28]:
engine.execute('DROP TABLE IF EXISTS combined_craft_data')
combined_df.to_sql('combined_craft_data', con=engine, if_exists='append', index=True)

In [29]:
result = engine.execute('SELECT * FROM combined_craft_data')
print(result)